<a href="https://colab.research.google.com/github/eduardoplima/decisoes-lm/blob/main/decisoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uso de LLMs para análise de decisões do Tribunal de Contas do Estado do Rio Grande do Norte

Os Tribunais de Contas possuem atribuições constitucionais diversas, que abrangem desde o julgamento de contas públicas à análise de atos de pessoal (tais como admissão e aposentadoria de servidores público). Seu colegiado, composto normalmente por sete conselheiros, tem o poder de determinar multas ou obrigações de fazer a gestores públicos responsáveis por órgãos jurisdicionados que descumpram normas legais ou conformidades operacionais.

No caso específico do Tribunal de Contas do Estado do Rio Grande do Norte, sua estrutura funcional, recém modificada, contempla uma diretoria cuja função é a gestão do exame, instrução e encaminhamentos referentes a processos de controle externo. Trata-se da Diretoria de Instrução Processual, cuja subunidade denominada Coordenadoria de Controle de Decisões (CCD) é responsável pelo cadastro, controle e monitoramento das obrigações decorrentes das decisões do TCE/RN e das recomendações de fiscalização e auditoria.

Também é da incumbência da CCD o criação do Cadastro Geral de Acompanhamento de Decisões (CGAD), conforme Art. 431 do Regimento Interno do TCE/RN. O referido cadastro inclui:

* Cadastro Geral de Multas (CGM), com acompanhamento permanente dos pagamentos
diretos ao Tribunal;
* Cadastro Geral de Devoluções (CGD), para acompanhamento permanente das condenações para devolução de valores ao Tesouro Estadual e aos Tesouros Municipais;
* Cadastro Geral de Recomendações (CGR), para acompanhamento permanente de todas as decisões para obrigação de fazer ou não fazer.
* Cadastro Geral de Termos de Ajustamento de Gestão (CGTAG), para acompanhamento dos Termos de Ajustamento de Gestão celebrados pelo Ministério Público junto ao Tribunal.

Esse notebook emprega técnicas de Reconhecimento de Entidades Nomeadas, _Word embeddings_ e LLMs para a criação dos cadastros mencionados.


In [ ]:
!pip install langchain_openai langchain_community gdown

In [2]:
import os
import getpass
import gdown

import pandas as pd

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OPENAI_API_KEY:")

OPENAI_API_KEY:··········


# Dataset

O dataset utilizado é composto por decisões exaradas pelo TCE/RN no ano de 2024. Os processos do TCE/RN são compostos por informações. Cada informação representa eventos de um processo, tais como uma petição inicial, um relatório de auditoria, a juntada de um documento, ou, como o caso em tela, a decisão sobre o processo. O conjunto de dados disponível é uma lista de objetos JSON criada a partir da junção das tabelas processo e informação do banco de dados administrativo do TCE/RN. Cada objeto contém os seguintes campos:

* `numero_processo`
* `ano_processo`
* `setor`: o setor de origem da decisão, nesse caso, o gabinete do conselheiro responsável pela decisão;
* `codigo_tipo_processo`: o tipo de processo sobre o qual a decisão foi determinada. Um dataframe com as descrições de cada tipo é criado abaixo;
* `assunto`: descrição simples do processo;
* `resumo`: um resumo da informação de decisão cadastrada;
* `texto`: o texto completo de cada decisão, esse campo é obtido por meio da leitura de um arquivo pdf.

Foram mantidos no dataset apenas processos referentes a atuações do TCE/RN no controle externo, excluindo-se as decisões referentes a atos de pessoal (aposentadoria, admissão e pensão), assuntos administrativos e "outros". Esses assunto são tratados por outras diretorias na estrutura organizacional do TCE/RN (por exemplo, a Diretoria de Atos de Pessoal). Ademais, para esse experimento, só foram escolhidos processos recentes (2021 em diante).

In [4]:
url = "https://raw.githubusercontent.com/eduardoplima/decisoes-lm/refs/heads/main/dataset.json"
output = "dataset.json"
gdown.download(url, output)

url = "https://raw.githubusercontent.com/eduardoplima/decisoes-lm/refs/heads/main/tipos_processos.csv"
output = "tipos_processos.csv"
gdown.download(url, output)

Downloading...
From: https://raw.githubusercontent.com/eduardoplima/decisoes-lm/refs/heads/main/dataset.json
To: /content/dataset.json
1.20MB [00:00, 108MB/s]                   
Downloading...
From: https://raw.githubusercontent.com/eduardoplima/decisoes-lm/refs/heads/main/tipos_processos.csv
To: /content/tipos_processos.csv
15.7kB [00:00, 19.3MB/s]                   


'tipos_processos.csv'

In [5]:
tipos = pd.read_csv("tipos_processos.csv")

In [6]:
df = pd.read_json("dataset.json")

In [7]:
len(df)

336

In [8]:
df.head()

,numero_processo,ano_processo,codigo_tipo_processo,assunto,setor,resumo,data_resumo,arquivo,texto
0,200090,2021,APR,INADIMPLÊNCIA NA REMESSA DE SIAI DP NOVO ...,secpc,ACÓRDÃO PADRÃO,2024-02-05,secpc_200090_2021_0043.pdf,[Processo Nº 200090 / 2021 - TC (200090/2021-T...
1,200161,2022,CGE,INADIMPLÊNCIA NA REMESSA DE CONTAS ANUAIS DE G...,secsc,ACÓRDÃO PADRÃO,2024-01-17,secsc_200161_2022_0021.pdf,[Processo Nº 200161 / 2022 - TC (200161/2022-C...
2,907,2022,ACO,ACOMPANHAMENTO DE ACUMULAÇÃO DE CARGOS PÚBLICO...,secsc,ACÓRDÃO PADRÃO,2024-02-07,secsc_000907_2022_0041.pdf,[Processo Nº 000907 / 2022 - TC (000907/2022-T...
3,1264,2022,REP,REPRESENTAÇÃO EM FACE DE CONTRATAÇÕES TEMPORÁR...,secpc,ACÓRDÃO PADRÃO,2024-02-08,secpc_001264_2022_0051.pdf,[Processo Nº 001264 / 2022 - TC (001264/2022-T...
4,3101,2022,DEN,xxx ...,secsc,ACÓRDÃO PADRÃO,2024-01-25,secsc_003101_2022_0015.pdf,[Processo Nº 003101 / 2022 - TC (003101/2022-T...


O texto vem separado em uma lista, cujos índices representam páginas do pdf. Nós agrupamos a string a seguir.

In [9]:
df['texto'] = df['texto'].apply(lambda x: ''.join(x))

Em uma análise preliminar percebe-se que as decisões seguem padrões bem diferentes. Por exemplo, abaixo se vê uma determinação de multa. Não há padronização nos campos, ou sequer uma indicação da categoria da decisão.

In [14]:
print(df.iloc[15].texto)

Processo Nº 200158 / 2021 - TC (200158/2021-TC)
SESSÃO ORDINÁRIA 00046ª, DE 19 DE DEZEMBRO DE 2023 - 2ª CÂMARA.
Interessado(s):   URBANA - COMPANHIA DE SERVIÇOS URBANOS DE NATAL, POR SEU ATUAL 
GESTOR - CPF:08498701000104
  
Assunto:  INADIMPLÊNCIA NA REMESSA DE CONTAS ANUAIS DE GESTÃO
Responsável(is):  Jonny Araujo da Costa - CPF:20119178400 Joseildes Medeiros da Silva - 
CPF:22127909453
Relator(a):  MARIA ADÉLIA SALES
EMENTA: ATRASO NA PRESTAÇÃO DAS CONTAS ANUAIS DE GESTÃO . 
PREJUÍZOS DIRETOS À ATIVIDADE FISCALIZATÓRIA DESTA CORTE . 
DEFESA INCONSISTENTE. ATRASO CONFIGURADO. IRREGULARIDADE DA 
MATÉRIA, NOS TERMOS DO ART. 75, I, DA LCE Nº 464/2012. MULTA AO 
GESTOR RESPONSÁVEL QUE SE ENCONTRA V A OBRIGADO A APRESENTAR 
AS CONTAS NA DATA DO SEU VENCIMENTO, NOS MOLDES DO ART. 107, II, 
“A” E “B”, DA LCE Nº 464/2012 C/C ART. 21, I, “B” E § 2º, DA RESOLUÇÃO Nº 
012/2016-TCE.
ACÓRDÃO No. 340/2023 - TC
               Vistos, relatados e discutidos estes autos,  concordando com o Corpo Inst

Abaixo um exemplo de obrigação.

In [15]:
print(df.iloc[20].texto)

Processo Nº 001231 / 2023 - TC (001231/2023-TC)
SESSÃO ORDINÁRIA 00004ª, DE 15 DE FEVEREIRO DE 2024 - 1ª CÂMARA.
Interessado(s):   PREFEITURA MUNICIPAL DE NÍSIA FLORESTA/RN                                                                                                                                                                                                                      
  
Assunto:  DENÚNCIA
Relator(a):  MARCO ANTÔNIO DE MORAES RÊGO MONTENEGRO
EMENTA: DENÚNCIA. AUSÊNCIA DE LASTRO PROBATÓRIO MÍNIMO PARA 
O DESENVOLVIMENTO REGULAR DO PROCESSO.  ADMISSIBILIDADE 
PREJUDICADA. INTELIGÊNCIA DO ARTIGO 80, § 1º, DA LOTCE . 
ARQUIV AMENTO DO PRESENTE FEITO.
ACÓRDÃO No. 40/2024 - TC
               Vistos, relatados e discutidos estes autos, em consonância ao posicionamento do 
Corpo técnico e do Ministério Público de Contas, ACORDAM os Conselheiros, nos termos 
do voto proposto pelo Conselheiro Relator, julgar a inadmissibilidade da presente denúncia e 
o seu conseqüente arquivamen

Por fim, abaixo vemos um exemplo de Termo de Ajustamento de Gestão. O instrumento também cria uma obrigação.

In [19]:
print(df[df.codigo_tipo_processo == 'TAG'].iloc[0].texto)

Processo Nº 002056 / 2023 - TC (002056/2023-TC)
SESSÃO ORDINÁRIA 00055ª, DE 06 DE AGOSTO DE 2024 - PLENO.
Interessado(s):   MINISTÉRIO PÚBLICO DE CONTAS DO RN                                                                                                                                                                                                                             
  
Assunto:  TERMO DE AJUSTAMENTO DE GESTÃO 001/2023
Relator(a):  PAULO ROBERTO CHA VES ALVES
EMENTA: TERMO DE AJUSTAMENTO DE GESTÃO. HOMOLOGAÇÃO. 
DEFERIMENTO. 
1. TERMO DE AJUSTAMENTO DE GESTÃO COM ESCOPO DE REALIZAÇÃO 
DE CONCURSO PÚBLICO PARA CRIAÇÃO E PROVIMENTO DE 180 (CENTO E 
OITENTA) CARGOS EFETIVOS NO IDEMA, EM SUBSTITUIÇÃO A AGENTES 
VINCULADOS DE FORMA PRECÁRIA POR MEIO DE BOLSA DE PESQUISA.
2. PREENCHIMENTO DOS REQUISITOS EXIGIDOS PELA LEI ORGÂNICA E 
REGIMENTO INTERNO. HOMOLOGAÇÃO NOS TERMOS DO ART. 122, §3º, DA 
LEI COMPLEMENTAR Nº 464/2012.
ACÓRDÃO No. 328/2024 - TC
               Vistos, relatado

array(['ACO', 'AGE', 'AOP', 'APR', 'AUD', 'CFM', 'CGE', 'DEN', 'EXE',
       'FIN', 'MON', 'PFA', 'REL', 'REP', 'TAG'], dtype=object)

# LLMs

Nessa seção configuramos os modelos de LLM que vamos utilizar nas seções seguintes

In [13]:
from langchain_openai import OpenAI
llm = OpenAI()

# Classificação de Texto e Reconhecimento de Entidades Nomeadas

O Reconhecimento de Entidades Nomeadas (Named Entity Recognition - NER) é uma técnica fundamental em Processamento de Linguagem Natural (PLN) que visa identificar e classificar automaticamente elementos chave em um texto, como nomes de pessoas, organizações, locais, datas, valores monetários, entre outros. Essa tarefa é essencial para o nosso objetivo, pois é imprescindível detectar os responsáveis citados e outras características da decisão.

A análise preliminar demonstrou a necessidade de primeiro definir se há uma multa ou obrigação envolvida na decisão. Após isso, devemos extrair os elementos pertinentes. Desse modo, vamos utilizar uma estratégia de agentes, onde cada um será responsável por tarefas diferentes na nossa linha de produção. Primeiro definimos o tipo do nosso texto e depois extraímos os elementos correspondentes a nossa classificação


In [ ]:
def is_multa_obrigação(texto):
